In [4]:
# instalo openpyxl para leer el Excel, matplotlib y seaborn (para los gráficos y análisis estadísticos) e importo las librerías de panda y numpys, 
# además de load_workbook desde openpyxl para que reconozca las 43.000 filas

!python -m pip install openpyxl matplotlib seaborn
from openpyxl import load_workbook
import pandas as pd
import numpy as np




In [9]:
# abro con openpyxl para asegurar que detectamos todas las filas

wb = load_workbook("Trabajo_excel_python.xlsx")
ws = wb.active
print(f"Filas totales detectadas por openpyxl: {ws.max_row}, Columnas: {ws.max_column}")


Filas totales detectadas por openpyxl: 43001, Columnas: 31


In [10]:
# cargo a pandas asegurándome de tomar todas las filas

df = pd.read_excel("Trabajo_excel_python.xlsx", engine="openpyxl", header=0)

In [116]:
# Creo generador de todas las filas
data = ws.values

# Tomo la primera fila como nombres de columnas
cols = next(data)

# Creo DataFrame con el resto de las filas
df = pd.DataFrame(data, columns=cols)

In [117]:
# reviso que cargan todas las filas

print(f"Dataset cargado en pandas: {df.shape[0]} filas x {df.shape[1]} columnas")


Dataset cargado en pandas: 43000 filas x 31 columnas


In [119]:
print("\n--- Información general ---")
print(df.info())

summary = pd.DataFrame({
    "dtype": df.dtypes,
    "n_nulls": df.isna().sum(),
    "n_unique": df.nunique()
})
print("\n--- Resumen de columnas ---")
print(summary)


--- Información general ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43000 entries, 0 to 42999
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Year                                 43000 non-null  int64         
 1   Age                                  37880 non-null  float64       
 2   Start date                           42752 non-null  datetime64[ns]
 3   Job                                  43000 non-null  object        
 4   Education                            43000 non-null  object        
 5   Marital                              43000 non-null  object        
 6   Income                               43000 non-null  int64         
 7   NumWebVisitsMonth                    43000 non-null  int64         
 8   Kidhome                              43000 non-null  int64         
 9   Teenhome                             43000 non-null  i

In [122]:
# separo columnas numéricas y categóricas

numeric_cols = df.select_dtypes(include=["number"]).columns
categorical_cols = df.select_dtypes(include=["object", "category"]).columns

print(f"\nColumnas numéricas ({len(numeric_cols)}): {list(numeric_cols)}")
print(f"Columnas categóricas ({len(categorical_cols)}): {list(categorical_cols)}")


Columnas numéricas (18): ['Year', 'Age', 'Income', 'NumWebVisitsMonth', 'Kidhome', 'Teenhome', 'Payment default', 'Current campaign calls', 'Days from last call', 'Previous contacts', 'Employment var.rate', 'Cons.price.idx', 'Cons.conf.idx', 'Euribor3months', 'Nr.employed', 'Latitude', 'Longitude', 'Duration']
Columnas categóricas (11): ['Job', 'Education', 'Marital', 'Housing', 'Loan', 'Service/product contracted', 'Previous marketing campaign outcome', 'Coordinates', 'ID', None, None]


In [129]:
# elimino las columnas sin nombre, con nombre NaN o con espacios, las inválidas y las que dan problemas:

df = df.loc[:, df.columns.notna()]
df = df.loc[:, df.columns.astype(str).str.strip() != ""]
bad_cols = [col for col in df.columns if col is None or str(col).strip() == ""]
categorical_cols = [
    col for col in categorical_cols
    if col is not None and col in df.columns
]


In [130]:
#estadísticas de las columnas numéricas

print("\n--- Estadísticas numéricas ---")
numeric_stats = df[numeric_cols].describe(percentiles=[0.25, 0.5, 0.75]).T
numeric_stats["median"] = df[numeric_cols].median()
print(numeric_stats)


--- Estadísticas numéricas ---
                          count          mean           std         min  \
Year                    43000.0   2012.861535      0.879460    2012.000   
Age                     37880.0     39.977112     10.437957      17.000   
Income                  43000.0  93241.200070  50498.316182    5841.000   
NumWebVisitsMonth       43000.0     16.589698      9.239205       1.000   
Kidhome                 43000.0      1.004791      0.815913       0.000   
Teenhome                43000.0      0.998605      0.815983       0.000   
Payment default         34019.0      0.000882      0.093905       0.000   
Current campaign calls  43000.0      2.567233      2.772294       1.000   
Days from last call     43000.0    962.330953    187.260394       0.000   
Previous contacts       43000.0      0.174023      0.497366       0.000   
Employment var.rate     43000.0      0.771279     15.738984     -34.000   
Cons.price.idx          42529.0     93.574219      0.579548      92.

In [131]:
# redefino las columnas categóricas:

categorical_cols = df.select_dtypes(include=["object", "category"]).columns

In [132]:
#estadísticas de las columnas categóricas

print("\n--- Estadísticas categóricas ---")
for col in categorical_cols:
    print(f"\nColumna: {col}")
    counts = df[col].value_counts(dropna=False)
    percents = df[col].value_counts(normalize=True, dropna=False) * 100
    cat_summary = pd.DataFrame({"count": counts, "percent": percents})
    print(cat_summary)


--- Estadísticas categóricas ---

Columna: Job
               count    percent
Job                            
admin.         10873  25.286047
blue-collar     9654  22.451163
technician      7026  16.339535
services        4162   9.679070
management      3050   7.093023
retired         1790   4.162791
entrepreneur    1522   3.539535
self-employed   1489   3.462791
housemaid       1123   2.611628
unemployed      1063   2.472093
student          903   2.100000
                 345   0.802326

Columna: Education
                     count    percent
Education                            
university.degree    12722  29.586047
high.school           9925  23.081395
basic.9y              6309  14.672093
professional.course   5477  12.737209
basic.4y              4356  10.130233
basic.6y              2386   5.548837
                      1807   4.202326
illiterate              18   0.041860

Columna: Marital
          count    percent
Marital                   
MARRIED   25999  60.462791
SINGL

In [140]:
# aunque en excel las pasé a Yes / No para que fueran más claras, vuelvo a pasar las columnas binarias a 1 y 0:

def binary_transform(series):
    s = series.copy()

    # Si ya es numérica (0/1)
    if pd.api.types.is_numeric_dtype(s):
        uniq = sorted(s.dropna().unique())
        if uniq == [0, 1]:
            return s

    # Normalizar texto
    s = s.astype(str).str.strip().str.lower()

    mapping = {
        "yes": 1, "no": 0,
        "y": 1, "n": 0,
        "true": 1, "false": 0,
        "1": 1, "0": 0,
        "sí": 1, "si": 1
    }

    mapped = s.map(mapping)

    if mapped.notna().sum() >= s.notna().sum() * 0.9:
        return mapped
    
for col in binary_cols:
    df[col] = binary_transform(df[col])

safe_binary_cols = []

# separo binarias seguras de dudosas
for col in binary_cols:
    vals = df[col].dropna().unique()
    if set(vals).issubset({0, 1}):
        safe_binary_cols.append(col)


In [141]:
# verifico datos transformados

for col in binary_cols:
    print(f"\n{col}")
    print(df[col].value_counts(dropna=False))


Payment default
Payment default
None    43000
Name: count, dtype: int64

Housing
Housing
1.0    22498
0.0    19476
NaN     1026
Name: count, dtype: int64

Loan
Loan
0.0    35442
1.0     6532
NaN     1026
Name: count, dtype: int64

Service/product contracted
Service/product contracted
0    38156
1     4844
Name: count, dtype: int64
